In [ ]:
# ================== CARBON FOOTPRINT CALCULATOR - GRADIO VERSION ==================
# This version uses Gradio which works reliably in Colab and creates a proper web interface

# Install Gradio if needed
import subprocess
import sys
try:
    import gradio as gr
except ImportError:
    print("Installing Gradio...")
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'gradio'])
    import gradio as gr

try:
    import pulp
except ImportError:
    print("Installing pulp...")
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'pulp==2.7.0'])
    import pulp

import pandas as pd
import numpy as np

# Configuration
BACKEND = "none"  # "none", "groq", or "openai"
GROQ_API_KEY = ""
GROQ_MODEL = "llama-3.1-8b-instant"

# Initialize AI backends
_openai = None
_groq = None

if BACKEND == "groq":
    try:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'groq'])
        from groq import Groq
        _groq = Groq(api_key=GROQ_API_KEY)
    except:
        pass

# Emission factors
EF = pd.DataFrame([
    ["electricity","grid_average_location_based","kWh",0.35,"placeholder"],
    ["electricity","market_based_contract_zero","kWh",0.00,"placeholder"],
    ["fuel","natural_gas","kWh",0.202,"placeholder"],
    ["fuel","diesel","liter",2.68,"placeholder"],
    ["fuel","gasoline","liter",2.31,"placeholder"],
    ["fuel","lpg","kg",3.00,"placeholder"],
    ["refrigerant","R-410A","kg_leak",2088.0,"placeholder"],
    ["refrigerant","R-134a","kg_leak",1430.0,"placeholder"],
    ["travel_flight","short_haul_economy","passenger_km",0.15,"placeholder"],
    ["travel_flight","long_haul_economy","passenger_km",0.11,"placeholder"],
    ["travel_train","average","passenger_km",0.02,"placeholder"],
    ["logistics_road","average_heavy","ton_km",0.08,"placeholder"],
    ["logistics_sea","container_ship","ton_km",0.01,"placeholder"],
    ["logistics_air","freighter","ton_km",0.50,"placeholder"],
    ["spend","packaging","EUR",0.5,"placeholder"],
    ["spend","metals","EUR",1.0,"placeholder"],
    ["spend","it_equipment","EUR",0.3,"placeholder"],
    ["spend","office_supplies","EUR",0.2,"placeholder"],
    ["water","mains","m3",0.3,"placeholder"],
    ["waste","mixed_landfill","ton",100.0,"placeholder"],
], columns=["category","subcategory","unit","ef_kgco2e_per_unit","notes"])

MEAS = pd.DataFrame([
    ["EE_LED","LED retrofit","efficiency",30000,-5000,120000,0,30.0,10,"LED"],
    ["EE_VSD","VSD on motors","efficiency",60000,-3000,80000,0,20.0,10,"VSD"],
    ["PV_Rooftop","Rooftop PV 200 kWp","renewables",180000,-1000,300000,0,90.0,20,"PV"],
    ["HP_MidTemp","Replace gas boiler with HP","fuel_switch",220000,4000,250000,0,70.0,15,"HP"],
    ["Fleet_EV","Electrify 10 vans","transport",250000,-6000,70000,10000,50.0,8,"EV"],
    ["R_LeakMgmt","Refrigerant leak mgmt","hvac",25000,0,0,0,15.0,7,"Leak mgmt"],
    ["Log_ModeShift","Road->Rail shift","logistics",20000,-2000,0,0,25.0,5,"Rail shift"],
], columns=["id","name","category","capex_eur","annual_opex_change_eur",
            "annual_energy_saving_kwh","annual_fuel_saving_liters","annual_co2e_saving_tons","lifetime_years","notes"])

# Helper functions
def ef_lookup(df, category, subcategory, fallback=None):
    row = df[(df['category']==category) & (df['subcategory']==subcategory)]
    if row.empty: return fallback
    return float(row.iloc[0]['ef_kgco2e_per_unit'])

def sf(x, default=0.0):
    try:
        if x in [None, ""]: return default
        return float(x)
    except: return default

def compute_emissions(inp, efdf=EF):
    elec_kwh = sf(inp.get('electricity_kwh')); green_kwh = min(sf(inp.get('green_electricity_kwh')), elec_kwh)
    ef_grid  = ef_lookup(efdf,'electricity','grid_average_location_based',0.35)
    ef_green = ef_lookup(efdf,'electricity','market_based_contract_zero',0.0)
    scope2_loc_kg = elec_kwh*ef_grid
    scope2_mkt_kg = green_kwh*ef_green + (elec_kwh-green_kwh)*ef_grid
    scope1_fuels_kg = (sf(inp.get('natural_gas_kwh'))*ef_lookup(efdf,'fuel','natural_gas',0.202) +
                       sf(inp.get('diesel_liters'))*ef_lookup(efdf,'fuel','diesel',2.68) +
                       sf(inp.get('gasoline_liters'))*ef_lookup(efdf,'fuel','gasoline',2.31) +
                       sf(inp.get('lpg_kg'))*ef_lookup(efdf,'fuel','lpg',3.0))
    scope1_ref_kg = (sf(inp.get('r410a_kg_leaked'))*ef_lookup(efdf,'refrigerant','R-410A',2088.0) +
                     sf(inp.get('r134a_kg_leaked'))*ef_lookup(efdf,'refrigerant','R-134a',1430.0))
    scope3_tl_kg = (sf(inp.get('flights_short_pkm'))*ef_lookup(efdf,'travel_flight','short_haul_economy',0.15) +
                    sf(inp.get('flights_long_pkm'))*ef_lookup(efdf,'travel_flight','long_haul_economy',0.11) +
                    sf(inp.get('train_pkm'))*ef_lookup(efdf,'travel_train','average',0.02) +
                    sf(inp.get('road_ton_km'))*ef_lookup(efdf,'logistics_road','average_heavy',0.08) +
                    sf(inp.get('sea_ton_km'))*ef_lookup(efdf,'logistics_sea','container_ship',0.01) +
                    sf(inp.get('air_ton_km'))*ef_lookup(efdf,'logistics_air','freighter',0.50))
    scope3_spend_kg = (sf(inp.get('spend_packaging_eur'))*ef_lookup(efdf,'spend','packaging',0.5) +
                       sf(inp.get('spend_metals_eur'))*ef_lookup(efdf,'spend','metals',1.0) +
                       sf(inp.get('spend_it_eur'))*ef_lookup(efdf,'spend','it_equipment',0.3) +
                       sf(inp.get('spend_office_eur'))*ef_lookup(efdf,'spend','office_supplies',0.2))
    scope3_ww_kg = (sf(inp.get('waste_mixed_ton'))*ef_lookup(efdf,'waste','mixed_landfill',100.0) +
                    sf(inp.get('water_m3'))*ef_lookup(efdf,'water','mains',0.3))
    totals = {
        'scope1_fuels_t': scope1_fuels_kg/1000.0, 'scope1_refrigerants_t': scope1_ref_kg/1000.0,
        'scope2_location_t': scope2_loc_kg/1000.0, 'scope2_market_t': scope2_mkt_kg/1000.0,
        'scope3_travel_logistics_t': scope3_tl_kg/1000.0, 'scope3_spend_t': scope3_spend_kg/1000.0,
        'scope3_waste_water_t': scope3_ww_kg/1000.0
    }
    totals['total_location_t'] = sum([v for k,v in totals.items() if k.endswith('_t') and k!='scope2_market_t'])
    totals['total_market_t'] = totals['total_location_t'] - totals['scope2_location_t'] + totals['scope2_market_t']
    return totals

def optimize_measures(measures_df, capex_budget, max_payback_yrs, val_e_kwh=0.12, val_fuel_l=1.7):
    ab = measures_df.copy()
    ab['annual_value_eur'] = -ab['annual_opex_change_eur'] + ab['annual_energy_saving_kwh']*val_e_kwh + ab['annual_fuel_saving_liters']*val_fuel_l
    ab['payback_years'] = ab['capex_eur'] / ab['annual_value_eur'].replace(0, np.nan)
    ab['eligible'] = ab['payback_years'] <= max_payback_yrs
    ab['co2e_saved_t'] = ab['annual_co2e_saving_tons']
    prob = pulp.LpProblem('AbatementSelection', pulp.LpMaximize)
    x = {row.id: pulp.LpVariable(f"x_{row.id}", lowBound=0, upBound=1, cat='Binary') for _, row in ab.iterrows()}
    prob += pulp.lpSum([x[row.id]*row.co2e_saved_t for _, row in ab.iterrows() if row.eligible]), "Max_CO2e"
    prob += pulp.lpSum([x[row.id]*row.capex_eur for _, row in ab.iterrows() if row.eligible]) <= capex_budget, "Budget"
    prob.solve(pulp.PULP_CBC_CMD(msg=False))
    sel = [row.to_dict() for _, row in ab.iterrows() if row.eligible and x[row.id].value()==1]
    return pd.DataFrame(sel), ab

def _fmt_t(v):
    try: return f"{float(v):,.1f}"
    except: return v

def build_emission_summary(totals):
    s1f = totals['scope1_fuels_t']
    s1r = totals['scope1_refrigerants_t']
    s2l = totals['scope2_location_t']
    s3t = totals['scope3_travel_logistics_t']
    s3s = totals['scope3_spend_t']
    s3w = totals['scope3_waste_water_t']
    tloc = totals['total_location_t']

    shares = []
    if tloc > 0:
        shares.append(["Scope 1 (Fuels)", _fmt_t(s1f), f"{100*s1f/tloc:.1f}%"])
        shares.append(["Scope 1 (Refrigerants)", _fmt_t(s1r), f"{100*s1r/tloc:.1f}%"])
        shares.append(["Scope 2 (Location)", _fmt_t(s2l), f"{100*s2l/tloc:.1f}%"])
        shares.append(["Scope 3 (Travel/Logistics)", _fmt_t(s3t), f"{100*s3t/tloc:.1f}%"])
        shares.append(["Scope 3 (Spend)", _fmt_t(s3s), f"{100*s3s/tloc:.1f}%"])
        shares.append(["Scope 3 (Waste/Water)", _fmt_t(s3w), f"{100*s3w/tloc:.1f}%"])

    return pd.DataFrame(shares, columns=["Category", "Emissions (tCO2e)", "% of Total"])

def make_recommendations(totals, selected, budget, payback):
    lines = []
    tloc = totals['total_location_t']
    if tloc > 0:
        s1 = totals['scope1_fuels_t'] + totals['scope1_refrigerants_t']
        s2 = totals['scope2_location_t']
        s3 = totals['scope3_travel_logistics_t'] + totals['scope3_spend_t'] + totals['scope3_waste_water_t']

        hotspots = sorted([("Scope 1", s1), ("Scope 2", s2), ("Scope 3", s3)], key=lambda x: x[1], reverse=True)
        lines.append(f"🎯 Top emissions source: {hotspots[0][0]} ({_fmt_t(hotspots[0][1])} tCO2e, {100*hotspots[0][1]/tloc:.1f}%)\n")

    if not selected.empty:
        lines.append(f"💡 {len(selected)} measures selected within €{budget:,.0f} budget:\n")
        for _, row in selected.iterrows():
            lines.append(f"  • {row['name']}: {_fmt_t(row['annual_co2e_saving_tons'])} tCO2e/year, payback {row.get('payback_years', 'N/A'):.1f} years")
    else:
        lines.append(f"⚠️ No measures found within budget (€{budget:,.0f}) and payback ({payback} years) constraints.")

    return "\n".join(lines)

# Main calculation function
def calculate_footprint(
    # Energy inputs
    electricity_kwh, has_green, green_kwh, nat_gas_kwh, diesel_l, gasoline_l, lpg_kg,
    # Refrigerants
    r410a_kg, r134a_kg,
    # Travel & Logistics
    flights_short, flights_long, train_pkm, road_tkm, sea_tkm, air_tkm,
    # Spend & Waste
    spend_pack, spend_metal, spend_it, spend_office, waste_ton, water_m3,
    # Intensity
    revenue, output_ton, area_m2,
    # Optimizer
    capex_budget, max_payback, elec_price, fuel_price
):
    # Validate inputs
    if (electricity_kwh == 0 and nat_gas_kwh == 0 and diesel_l == 0 and
        flights_short == 0 and road_tkm == 0):
        return "⚠️ Please enter at least one non-zero activity.", None, ""

    # Build input dict
    inputs = {
        'electricity_kwh': electricity_kwh,
        'green_electricity_kwh': green_kwh if has_green else 0,
        'natural_gas_kwh': nat_gas_kwh,
        'diesel_liters': diesel_l,
        'gasoline_liters': gasoline_l,
        'lpg_kg': lpg_kg,
        'r410a_kg_leaked': r410a_kg,
        'r134a_kg_leaked': r134a_kg,
        'flights_short_pkm': flights_short,
        'flights_long_pkm': flights_long,
        'train_pkm': train_pkm,
        'road_ton_km': road_tkm,
        'sea_ton_km': sea_tkm,
        'air_ton_km': air_tkm,
        'spend_packaging_eur': spend_pack,
        'spend_metals_eur': spend_metal,
        'spend_it_eur': spend_it,
        'spend_office_eur': spend_office,
        'waste_mixed_ton': waste_ton,
        'water_m3': water_m3,
        'revenue_eur': revenue,
        'production_output_ton': output_ton,
        'facility_area_m2': area_m2,
    }

    # Calculate
    totals = compute_emissions(inputs, EF)
    selected, _ = optimize_measures(MEAS, capex_budget, max_payback, elec_price, fuel_price)
    shares_df = build_emission_summary(totals)

    # Format output
    total_loc = totals['total_location_t']
    total_mkt = totals['total_market_t']

    summary = f"""
📊 **TOTAL CARBON FOOTPRINT**
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Location-based total: **{_fmt_t(total_loc)} tCO₂e**
Market-based total:   **{_fmt_t(total_mkt)} tCO₂e**
"""

    recommendations = "\n💡 **RECOMMENDATIONS**\n" + "━" * 60 + "\n" + make_recommendations(totals, selected, capex_budget, max_payback)

    return summary, shares_df, recommendations

# Fill example data function
def fill_example():
    return (
        800000, True, 0, 400000, 50000, 10000, 0,  # Energy
        5, 0,  # Refrigerants
        120000, 80000, 15000, 900000, 0, 5000,  # Travel
        250000, 500000, 80000, 40000, 120, 2000,  # Spend
        12000000, 1500, 6000,  # Intensity
        500000, 5, 0.12, 1.7  # Optimizer
    )

# Create Gradio interface
with gr.Blocks(title="Carbon Footprint Calculator", theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🌍 Carbon Footprint Calculator")
    gr.Markdown("Calculate your organization's carbon emissions and get actionable recommendations.")

    with gr.Row():
        fill_btn = gr.Button("📝 Fill Example Data", variant="secondary")
        calc_btn = gr.Button("▶️ Calculate Footprint", variant="primary", scale=2)

    with gr.Tabs():
        with gr.TabItem("⚡ Energy"):
            with gr.Row():
                electricity_kwh = gr.Number(label="Electricity (kWh)", value=0)
                has_green = gr.Checkbox(label="Green contract?", value=False)
                green_kwh = gr.Number(label="Green electricity (kWh)", value=0)
            with gr.Row():
                nat_gas_kwh = gr.Number(label="Natural Gas (kWh)", value=0)
                diesel_l = gr.Number(label="Diesel (L)", value=0)
                gasoline_l = gr.Number(label="Gasoline (L)", value=0)
                lpg_kg = gr.Number(label="LPG (kg)", value=0)

        with gr.TabItem("❄️ Refrigerants"):
            with gr.Row():
                r410a_kg = gr.Number(label="R-410A leaked (kg)", value=0)
                r134a_kg = gr.Number(label="R-134a leaked (kg)", value=0)

        with gr.TabItem("✈️ Travel & Logistics"):
            with gr.Row():
                flights_short = gr.Number(label="Short haul flights (passenger·km)", value=0)
                flights_long = gr.Number(label="Long haul flights (passenger·km)", value=0)
                train_pkm = gr.Number(label="Train (passenger·km)", value=0)
            with gr.Row():
                road_tkm = gr.Number(label="Road freight (ton·km)", value=0)
                sea_tkm = gr.Number(label="Sea freight (ton·km)", value=0)
                air_tkm = gr.Number(label="Air freight (ton·km)", value=0)

        with gr.TabItem("📦 Spend, Waste & Water"):
            with gr.Row():
                spend_pack = gr.Number(label="Packaging spend (EUR)", value=0)
                spend_metal = gr.Number(label="Metals spend (EUR)", value=0)
                spend_it = gr.Number(label="IT equipment (EUR)", value=0)
                spend_office = gr.Number(label="Office supplies (EUR)", value=0)
            with gr.Row():
                waste_ton = gr.Number(label="Waste (ton)", value=0)
                water_m3 = gr.Number(label="Water (m³)", value=0)

        with gr.TabItem("📊 Intensity & Context"):
            with gr.Row():
                revenue = gr.Number(label="Revenue (EUR)", value=0)
                output_ton = gr.Number(label="Production output (ton)", value=0)
                area_m2 = gr.Number(label="Facility area (m²)", value=0)

        with gr.TabItem("💰 Optimizer Settings"):
            with gr.Row():
                capex_budget = gr.Number(label="CAPEX Budget (EUR)", value=0)
                max_payback = gr.Number(label="Max payback (years)", value=5)
            with gr.Row():
                elec_price = gr.Number(label="Electricity price (EUR/kWh)", value=0.12)
                fuel_price = gr.Number(label="Fuel price (EUR/L)", value=1.7)

    # Outputs
    gr.Markdown("---")
    summary_out = gr.Markdown()
    breakdown_out = gr.Dataframe(headers=["Category", "Emissions (tCO2e)", "% of Total"])
    recommendations_out = gr.Markdown()

    # Wire up buttons
    inputs = [
        electricity_kwh, has_green, green_kwh, nat_gas_kwh, diesel_l, gasoline_l, lpg_kg,
        r410a_kg, r134a_kg,
        flights_short, flights_long, train_pkm, road_tkm, sea_tkm, air_tkm,
        spend_pack, spend_metal, spend_it, spend_office, waste_ton, water_m3,
        revenue, output_ton, area_m2,
        capex_budget, max_payback, elec_price, fuel_price
    ]

    calc_btn.click(
        fn=calculate_footprint,
        inputs=inputs,
        outputs=[summary_out, breakdown_out, recommendations_out]
    )

    fill_btn.click(
        fn=fill_example,
        inputs=None,
        outputs=inputs
    )

# Launch the app
print("🚀 Launching Carbon Footprint Calculator...")
demo.launch(share=True, debug=True)

🚀 Launching Carbon Footprint Calculator...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://3baf9ce337ca6eaa4f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://3baf9ce337ca6eaa4f.gradio.live


In [ ]:
# ================== CARBON FOOTPRINT CALCULATOR - WORKING COLAB VERSION =============

import json
import pandas as pd
import numpy as np
from IPython.display import display, HTML, Javascript

# Install dependencies
try:
    import pulp
except ImportError:
    import subprocess
    import sys
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'pulp==2.7.0'])
    import pulp

print("=" * 70)
print("⚙️  Carbon Footprint Calculator")
print("=" * 70)
print("📝 To enable AI recommendations (optional):")
print("   1. Get a free API key from: https://console.groq.com")
print("   2. Enter it in the '🤖 AI Settings' tab")
print("=" * 70)

# Configuration
GROQ_MODEL = "llama-3.1-8b-instant"

# Emission factors
EF = pd.DataFrame([
    ["electricity","grid_average_location_based","kWh",0.35,"placeholder"],
    ["electricity","market_based_contract_zero","kWh",0.00,"placeholder"],
    ["fuel","natural_gas","kWh",0.202,"placeholder"],
    ["fuel","diesel","liter",2.68,"placeholder"],
    ["fuel","gasoline","liter",2.31,"placeholder"],
    ["fuel","lpg","kg",3.00,"placeholder"],
    ["refrigerant","R-410A","kg_leak",2088.0,"placeholder"],
    ["refrigerant","R-134a","kg_leak",1430.0,"placeholder"],
    ["travel_flight","short_haul_economy","passenger_km",0.15,"placeholder"],
    ["travel_flight","long_haul_economy","passenger_km",0.11,"placeholder"],
    ["travel_train","average","passenger_km",0.02,"placeholder"],
    ["logistics_road","average_heavy","ton_km",0.08,"placeholder"],
    ["logistics_sea","container_ship","ton_km",0.01,"placeholder"],
    ["logistics_air","freighter","ton_km",0.50,"placeholder"],
    ["spend","packaging","EUR",0.5,"placeholder"],
    ["spend","metals","EUR",1.0,"placeholder"],
    ["spend","it_equipment","EUR",0.3,"placeholder"],
    ["spend","office_supplies","EUR",0.2,"placeholder"],
    ["water","mains","m3",0.3,"placeholder"],
    ["waste","mixed_landfill","ton",100.0,"placeholder"],
], columns=["category","subcategory","unit","ef_kgco2e_per_unit","notes"])

MEAS = pd.DataFrame([
    ["EE_LED","LED retrofit","efficiency",30000,-5000,120000,0,30.0,10,"LED"],
    ["EE_VSD","VSD on motors","efficiency",60000,-3000,80000,0,20.0,10,"VSD"],
    ["PV_Rooftop","Rooftop PV 200 kWp","renewables",180000,-1000,300000,0,90.0,20,"PV"],
    ["HP_MidTemp","Replace gas boiler with HP","fuel_switch",220000,4000,250000,0,70.0,15,"HP"],
    ["Fleet_EV","Electrify 10 vans","transport",250000,-6000,70000,10000,50.0,8,"EV"],
    ["R_LeakMgmt","Refrigerant leak mgmt","hvac",25000,0,0,0,15.0,7,"Leak mgmt"],
    ["Log_ModeShift","Road->Rail shift","logistics",20000,-2000,0,0,25.0,5,"Rail shift"],
], columns=["id","name","category","capex_eur","annual_opex_change_eur",
            "annual_energy_saving_kwh","annual_fuel_saving_liters","annual_co2e_saving_tons","lifetime_years","notes"])

# Helper functions
def ef_lookup(df, category, subcategory, fallback=None):
    row = df[(df['category']==category) & (df['subcategory']==subcategory)]
    if row.empty: return fallback
    return float(row.iloc[0]['ef_kgco2e_per_unit'])

def sf(x, default=0.0):
    try:
        if x in [None, ""]: return default
        return float(x)
    except: return default

def compute_emissions(inp, efdf=EF):
    elec_kwh = sf(inp.get('electricity_kwh')); green_kwh = min(sf(inp.get('green_electricity_kwh')), elec_kwh)
    ef_grid  = ef_lookup(efdf,'electricity','grid_average_location_based',0.35)
    ef_green = ef_lookup(efdf,'electricity','market_based_contract_zero',0.0)
    scope2_loc_kg = elec_kwh*ef_grid
    scope2_mkt_kg = green_kwh*ef_green + (elec_kwh-green_kwh)*ef_grid
    scope1_fuels_kg = (sf(inp.get('natural_gas_kwh'))*ef_lookup(efdf,'fuel','natural_gas',0.202) +
                       sf(inp.get('diesel_liters'))*ef_lookup(efdf,'fuel','diesel',2.68) +
                       sf(inp.get('gasoline_liters'))*ef_lookup(efdf,'fuel','gasoline',2.31) +
                       sf(inp.get('lpg_kg'))*ef_lookup(efdf,'fuel','lpg',3.0))
    scope1_ref_kg = (sf(inp.get('r410a_kg_leaked'))*ef_lookup(efdf,'refrigerant','R-410A',2088.0) +
                     sf(inp.get('r134a_kg_leaked'))*ef_lookup(efdf,'refrigerant','R-134a',1430.0))
    scope3_tl_kg = (sf(inp.get('flights_short_pkm'))*ef_lookup(efdf,'travel_flight','short_haul_economy',0.15) +
                    sf(inp.get('flights_long_pkm'))*ef_lookup(efdf,'travel_flight','long_haul_economy',0.11) +
                    sf(inp.get('train_pkm'))*ef_lookup(efdf,'travel_train','average',0.02) +
                    sf(inp.get('road_ton_km'))*ef_lookup(efdf,'logistics_road','average_heavy',0.08) +
                    sf(inp.get('sea_ton_km'))*ef_lookup(efdf,'logistics_sea','container_ship',0.01) +
                    sf(inp.get('air_ton_km'))*ef_lookup(efdf,'logistics_air','freighter',0.50))
    scope3_spend_kg = (sf(inp.get('spend_packaging_eur'))*ef_lookup(efdf,'spend','packaging',0.5) +
                       sf(inp.get('spend_metals_eur'))*ef_lookup(efdf,'spend','metals',1.0) +
                       sf(inp.get('spend_it_eur'))*ef_lookup(efdf,'spend','it_equipment',0.3) +
                       sf(inp.get('spend_office_eur'))*ef_lookup(efdf,'spend','office_supplies',0.2))
    scope3_ww_kg = (sf(inp.get('waste_mixed_ton'))*ef_lookup(efdf,'waste','mixed_landfill',100.0) +
                    sf(inp.get('water_m3'))*ef_lookup(efdf,'water','mains',0.3))
    totals = {
        'scope1_fuels_t': scope1_fuels_kg/1000.0, 'scope1_refrigerants_t': scope1_ref_kg/1000.0,
        'scope2_location_t': scope2_loc_kg/1000.0, 'scope2_market_t': scope2_mkt_kg/1000.0,
        'scope3_travel_logistics_t': scope3_tl_kg/1000.0, 'scope3_spend_t': scope3_spend_kg/1000.0,
        'scope3_waste_water_t': scope3_ww_kg/1000.0
    }
    totals['total_location_t'] = sum([v for k,v in totals.items() if k.endswith('_t') and k!='scope2_market_t'])
    totals['total_market_t'] = totals['total_location_t'] - totals['scope2_location_t'] + totals['scope2_market_t']
    return totals

def optimize_measures(measures_df, capex_budget, max_payback_yrs, val_e_kwh=0.12, val_fuel_l=1.7):
    ab = measures_df.copy()
    ab['annual_value_eur'] = -ab['annual_opex_change_eur'] + ab['annual_energy_saving_kwh']*val_e_kwh + ab['annual_fuel_saving_liters']*val_fuel_l
    ab['payback_years'] = ab['capex_eur'] / ab['annual_value_eur'].replace(0, np.nan)
    ab['eligible'] = ab['payback_years'] <= max_payback_yrs
    ab['co2e_saved_t'] = ab['annual_co2e_saving_tons']
    prob = pulp.LpProblem('AbatementSelection', pulp.LpMaximize)
    x = {row.id: pulp.LpVariable(f"x_{row.id}", lowBound=0, upBound=1, cat='Binary') for _, row in ab.iterrows()}
    prob += pulp.lpSum([x[row.id]*row.co2e_saved_t for _, row in ab.iterrows() if row.eligible]), "Max_CO2e"
    prob += pulp.lpSum([x[row.id]*row.capex_eur for _, row in ab.iterrows() if row.eligible]) <= capex_budget, "Budget"
    prob.solve(pulp.PULP_CBC_CMD(msg=False))
    sel = [row.to_dict() for _, row in ab.iterrows() if row.eligible and x[row.id].value()==1]
    return pd.DataFrame(sel), ab

def _fmt_t(v):
    try: return f"{float(v):,.1f}"
    except: return str(v)

def build_emission_summary(totals):
    s1f = totals['scope1_fuels_t']
    s1r = totals['scope1_refrigerants_t']
    s2l = totals['scope2_location_t']
    s3t = totals['scope3_travel_logistics_t']
    s3s = totals['scope3_spend_t']
    s3w = totals['scope3_waste_water_t']
    tloc = totals['total_location_t']

    shares = []
    if tloc > 0:
        shares.append(["Scope 1 (Fuels)", _fmt_t(s1f), f"{100*s1f/tloc:.1f}%"])
        shares.append(["Scope 1 (Refrigerants)", _fmt_t(s1r), f"{100*s1r/tloc:.1f}%"])
        shares.append(["Scope 2 (Location)", _fmt_t(s2l), f"{100*s2l/tloc:.1f}%"])
        shares.append(["Scope 3 (Travel/Logistics)", _fmt_t(s3t), f"{100*s3t/tloc:.1f}%"])
        shares.append(["Scope 3 (Spend)", _fmt_t(s3s), f"{100*s3s/tloc:.1f}%"])
        shares.append(["Scope 3 (Waste/Water)", _fmt_t(s3w), f"{100*s3w/tloc:.1f}%"])

    return shares, tloc, totals['total_market_t']

def make_recommendations(totals, selected, budget, payback):
    lines = []
    tloc = totals['total_location_t']
    if tloc > 0:
        s1 = totals['scope1_fuels_t'] + totals['scope1_refrigerants_t']
        s2 = totals['scope2_location_t']
        s3 = totals['scope3_travel_logistics_t'] + totals['scope3_spend_t'] + totals['scope3_waste_water_t']

        hotspots = sorted([("Scope 1", s1), ("Scope 2", s2), ("Scope 3", s3)], key=lambda x: x[1], reverse=True)
        lines.append(f"<h4>🎯 Top Emissions Source</h4>")
        lines.append(f"<p><strong>{hotspots[0][0]}</strong>: {_fmt_t(hotspots[0][1])} tCO2e ({100*hotspots[0][1]/tloc:.1f}% of total)</p>")

        if len(hotspots) > 1:
            lines.append(f"<p><strong>{hotspots[1][0]}</strong>: {_fmt_t(hotspots[1][1])} tCO2e ({100*hotspots[1][1]/tloc:.1f}%)</p>")
            lines.append(f"<p><strong>{hotspots[2][0]}</strong>: {_fmt_t(hotspots[2][1])} tCO2e ({100*hotspots[2][1]/tloc:.1f}%)</p>")

    if not selected.empty:
        lines.append(f"<h4>💡 Selected Reduction Measures</h4>")
        lines.append(f"<p><strong>{len(selected)} measures</strong> selected within €{budget:,.0f} budget</p>")
        total_reduction = selected['annual_co2e_saving_tons'].sum()
        total_capex = selected['capex_eur'].sum()
        lines.append(f"<ul>")
        lines.append(f"<li><strong>Total CO2e reduction</strong>: {_fmt_t(total_reduction)} tCO2e/year ({100*total_reduction/tloc:.1f}% of emissions)</li>")
        lines.append(f"<li><strong>Total CAPEX required</strong>: €{total_capex:,.0f}</li>")
        lines.append(f"</ul>")
        lines.append("<h5>Recommended projects:</h5><ul>")
        for _, row in selected.iterrows():
            pb = row.get('payback_years', 'N/A')
            pb_str = f"{pb:.1f}" if isinstance(pb, (int, float)) and not np.isnan(pb) else "N/A"
            lines.append(f"<li><strong>{row['name']}</strong>: {_fmt_t(row['annual_co2e_saving_tons'])} tCO2e/year | €{row['capex_eur']:,.0f} CAPEX | {pb_str} years payback</li>")
        lines.append("</ul>")
    else:
        lines.append(f"<h4>⚠️ No Measures Selected</h4>")
        lines.append(f"<p>No measures found within budget (€{budget:,.0f}) and max payback ({payback} years).</p>")
        lines.append("<p><strong>Suggestions:</strong></p><ul><li>Increase CAPEX budget</li><li>Extend max payback period</li><li>Review available measures</li></ul>")

    return "".join(lines)

# Global variable to store the last result
_last_result = {}

def calculate_and_store(data_json):
    """Calculate and store result globally"""
    global _last_result

    try:
        data = json.loads(data_json)
        print("🔄 Calculating carbon footprint...")
        print(f"📊 Received data: electricity={data.get('electricity_kwh', 0)} kWh")

        # Validate
        if (sf(data.get('electricity_kwh')) == 0 and sf(data.get('natural_gas_kwh')) == 0 and
            sf(data.get('diesel_liters')) == 0 and sf(data.get('flights_short_pkm')) == 0 and
            sf(data.get('road_ton_km')) == 0):
            _last_result = {'error': '⚠️ Please enter at least one non-zero activity value'}
            return

        # Calculate
        totals = compute_emissions(data, EF)
        print(f"✓ Total emissions: {_fmt_t(totals['total_location_t'])} tCO2e")

        selected, _ = optimize_measures(
            MEAS,
            sf(data.get('capex_budget')),
            sf(data.get('max_payback', 5)),
            sf(data.get('elec_price', 0.12)),
            sf(data.get('fuel_price', 1.7))
        )
        print(f"✓ Selected {len(selected)} reduction measures")

        shares, total_loc, total_mkt = build_emission_summary(totals)
        recommendations = make_recommendations(totals, selected, sf(data.get('capex_budget')), sf(data.get('max_payback', 5)))

        # AI recommendations (optional)
        ai_recs = ""
        if data.get('use_ai') and data.get('groq_api_key'):
            print("🤖 Generating AI recommendations...")
            try:
                try:
                    from groq import Groq
                except ImportError:
                    import subprocess
                    import sys
                    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'groq'])
                    from groq import Groq

                client = Groq(api_key=data['groq_api_key'])
                context = f"""Total emissions: {_fmt_t(total_loc)} tCO2e
Scope 1: {_fmt_t(totals['scope1_fuels_t'] + totals['scope1_refrigerants_t'])} tCO2e
Scope 2: {_fmt_t(totals['scope2_location_t'])} tCO2e
Scope 3: {_fmt_t(totals['scope3_travel_logistics_t'] + totals['scope3_spend_t'] + totals['scope3_waste_water_t'])} tCO2e
Selected {len(selected)} reduction measures within €{data.get('capex_budget', 0):,.0f} budget"""

                response = client.chat.completions.create(
                    model=GROQ_MODEL,
                    messages=[
                        {"role": "system", "content": "You are a carbon footprint advisor. Provide 3-5 prioritized, actionable recommendations."},
                        {"role": "user", "content": f"{context}\n\nProvide prioritized recommendations."}
                    ],
                    max_tokens=600
                )
                ai_recs = response.choices[0].message.content.strip().replace('\n', '<br>')
                print("✓ AI recommendations generated")
            except Exception as e:
                print(f"⚠️ AI error: {str(e)}")
                ai_recs = f"<span style='color: #856404;'>⚠️ AI unavailable: {str(e)}</span>"

        _last_result = {
            'success': True,
            'total_location': _fmt_t(total_loc),
            'total_market': _fmt_t(total_mkt),
            'breakdown': shares,
            'recommendations': recommendations,
            'ai_recommendations': ai_recs if ai_recs else None
        }
        print("✅ Calculation complete!")

    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()
        _last_result = {'error': f'Error: {str(e)}'}

# HTML Interface with simpler communication
html_code = """
<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <style>
        * { box-sizing: border-box; margin: 0; padding: 0; }
        body { font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Arial, sans-serif;
               background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
               padding: 20px; min-height: 100vh; }
        .container { max-width: 1200px; margin: 0 auto; background: white;
                     border-radius: 16px; box-shadow: 0 20px 60px rgba(0,0,0,0.3);
                     overflow: hidden; }
        .header { background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                  color: white; padding: 30px; text-align: center; }
        .header h1 { font-size: 36px; margin-bottom: 10px; }
        .header p { font-size: 18px; opacity: 0.9; }
        .controls { padding: 20px; background: #f8f9fa; border-bottom: 2px solid #dee2e6;
                    display: flex; gap: 10px; flex-wrap: wrap; justify-content: center; }
        .btn { padding: 12px 24px; border: none; border-radius: 8px; font-size: 16px;
               cursor: pointer; transition: all 0.3s; font-weight: 600; }
        .btn-primary { background: #667eea; color: white; }
        .btn-primary:hover { background: #5568d3; transform: translateY(-2px); }
        .btn-secondary { background: #6c757d; color: white; }
        .btn-secondary:hover { background: #5a6268; }
        .tabs { display: flex; border-bottom: 2px solid #dee2e6; background: #f8f9fa;
                padding: 0 20px; overflow-x: auto; }
        .tab { padding: 16px 24px; cursor: pointer; border-bottom: 3px solid transparent;
               transition: all 0.3s; white-space: nowrap; font-weight: 500; }
        .tab:hover { background: #e9ecef; }
        .tab.active { border-bottom-color: #667eea; color: #667eea; background: white; }
        .tab-content { display: none; padding: 30px; }
        .tab-content.active { display: block; }
        .form-group { margin-bottom: 20px; }
        .form-group label { display: block; margin-bottom: 8px; font-weight: 600;
                            color: #495057; font-size: 14px; }
        .form-group input[type="number"], .form-group input[type="text"] {
            width: 100%; padding: 12px; border: 2px solid #dee2e6;
            border-radius: 8px; font-size: 16px; transition: border 0.3s;
        }
        .form-group input:focus { outline: none; border-color: #667eea; }
        .form-group input[type="checkbox"] { width: 20px; height: 20px; cursor: pointer; }
        .form-row { display: grid; grid-template-columns: repeat(auto-fit, minmax(250px, 1fr));
                    gap: 20px; }
        .results { display: none; padding: 30px; background: #f8f9fa; border-top: 2px solid #dee2e6; }
        .results.show { display: block; }
        .loading { text-align: center; padding: 40px; font-size: 18px; color: #667eea; }
        .spinner { border: 4px solid #f3f3f3; border-top: 4px solid #667eea;
                   border-radius: 50%; width: 40px; height: 40px;
                   animation: spin 1s linear infinite; margin: 0 auto 20px; }
        @keyframes spin { 0% { transform: rotate(0deg); } 100% { transform: rotate(360deg); } }
        .results-summary { background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                           color: white; padding: 24px; border-radius: 12px; margin-bottom: 24px; }
        .results-summary h3 { margin-bottom: 16px; font-size: 24px; }
        .results-summary p { font-size: 18px; margin: 8px 0; }
        .results-table { width: 100%; border-collapse: collapse; margin: 20px 0;
                         background: white; border-radius: 8px; overflow: hidden; }
        .results-table th, .results-table td { padding: 16px; text-align: left;
                                                border-bottom: 1px solid #dee2e6; }
        .results-table th { background: #667eea; color: white; font-weight: 600; }
        .results-table tr:hover { background: #f8f9fa; }
        .recommendations { background: white; padding: 24px; border-radius: 12px;
                           margin: 20px 0; border-left: 4px solid #667eea; }
        .recommendations h4 { color: #667eea; margin-top: 16px; }
        .recommendations ul { margin-left: 20px; margin-top: 12px; }
        .recommendations li { margin: 8px 0; line-height: 1.6; }
        .error { background: #f8d7da; color: #721c24; padding: 16px;
                 border-radius: 8px; border-left: 4px solid #f5c6cb; }
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1>🌍 Carbon Footprint Calculator</h1>
            <p>Calculate your organization's emissions and get actionable recommendations</p>
        </div>

        <div class="controls">
            <button class="btn btn-secondary" onclick="fillExample()">📝 Fill Example Data</button>
            <button class="btn btn-secondary" onclick="clearForm()">🗑️ Clear All</button>
            <button class="btn btn-primary" onclick="calculateFootprint()">▶️ Calculate Footprint</button>
        </div>

        <div class="tabs">
            <div class="tab active" onclick="switchTab(0)">⚡ Energy</div>
            <div class="tab" onclick="switchTab(1)">❄️ Refrigerants</div>
            <div class="tab" onclick="switchTab(2)">✈️ Travel</div>
            <div class="tab" onclick="switchTab(3)">📦 Spend/Waste</div>
            <div class="tab" onclick="switchTab(4)">💰 Optimizer</div>
            <div class="tab" onclick="switchTab(5)">🤖 AI</div>
        </div>

        <div class="tab-content active">
            <h2>⚡ Energy & Fuels</h2>
            <div class="form-row">
                <div class="form-group">
                    <label for="electricity_kwh">Electricity (kWh/year)</label>
                    <input type="number" id="electricity_kwh" value="0" min="0">
                </div>
                <div class="form-group">
                    <label><input type="checkbox" id="has_green"> Green contract?</label>
                </div>
                <div class="form-group">
                    <label for="green_electricity_kwh">Green electricity (kWh/year)</label>
                    <input type="number" id="green_electricity_kwh" value="0" min="0">
                </div>
            </div>
            <div class="form-row">
                <div class="form-group">
                    <label for="natural_gas_kwh">Natural Gas (kWh/year)</label>
                    <input type="number" id="natural_gas_kwh" value="0" min="0">
                </div>
                <div class="form-group">
                    <label for="diesel_liters">Diesel (L/year)</label>
                    <input type="number" id="diesel_liters" value="0" min="0">
                </div>
                <div class="form-group">
                    <label for="gasoline_liters">Gasoline (L/year)</label>
                    <input type="number" id="gasoline_liters" value="0" min="0">
                </div>
                <div class="form-group">
                    <label for="lpg_kg">LPG (kg/year)</label>
                    <input type="number" id="lpg_kg" value="0" min="0">
                </div>
            </div>
        </div>

        <div class="tab-content">
            <h2>❄️ Refrigerants</h2>
            <div class="form-row">
                <div class="form-group">
                    <label for="r410a_kg_leaked">R-410A leaked (kg/year)</label>
                    <input type="number" id="r410a_kg_leaked" value="0" min="0">
                </div>
                <div class="form-group">
                    <label for="r134a_kg_leaked">R-134a leaked (kg/year)</label>
                    <input type="number" id="r134a_kg_leaked" value="0" min="0">
                </div>
            </div>
        </div>

        <div class="tab-content">
            <h2>✈️ Travel & Logistics</h2>
            <div class="form-row">
                <div class="form-group">
                    <label for="flights_short_pkm">Short flights (passenger·km)</label>
                    <input type="number" id="flights_short_pkm" value="0" min="0">
                </div>
                <div class="form-group">
                    <label for="flights_long_pkm">Long flights (passenger·km)</label>
                    <input type="number" id="flights_long_pkm" value="0" min="0">
                </div>
                <div class="form-group">
                    <label for="train_pkm">Train (passenger·km)</label>
                    <input type="number" id="train_pkm" value="0" min="0">
                </div>
            </div>
            <div class="form-row">
                <div class="form-group">
                    <label for="road_ton_km">Road freight (ton·km)</label>
                    <input type="number" id="road_ton_km" value="0" min="0">
                </div>
                <div class="form-group">
                    <label for="sea_ton_km">Sea freight (ton·km)</label>
                    <input type="number" id="sea_ton_km" value="0" min="0">
                </div>
                <div class="form-group">
                    <label for="air_ton_km">Air freight (ton·km)</label>
                    <input type="number" id="air_ton_km" value="0" min="0">
                </div>
            </div>
        </div>

        <div class="tab-content">
            <h2>📦 Spend, Waste & Water</h2>
            <div class="form-row">
                <div class="form-group">
                    <label for="spend_packaging_eur">Packaging (EUR/year)</label>
                    <input type="number" id="spend_packaging_eur" value="0" min="0">
                </div>
                <div class="form-group">
                    <label for="spend_metals_eur">Metals (EUR/year)</label>
                    <input type="number" id="spend_metals_eur" value="0" min="0">
                </div>
                <div class="form-group">
                    <label for="spend_it_eur">IT equipment (EUR/year)</label>
                    <input type="number" id="spend_it_eur" value="0" min="0">
                </div>
                <div class="form-group">
                    <label for="spend_office_eur">Office supplies (EUR/year)</label>
                    <input type="number" id="spend_office_eur" value="0" min="0">
                </div>
            </div>
            <div class="form-row">
                <div class="form-group">
                    <label for="waste_mixed_ton">Waste (ton/year)</label>
                    <input type="number" id="waste_mixed_ton" value="0" min="0">
                </div>
                <div class="form-group">
                    <label for="water_m3">Water (m³/year)</label>
                    <input type="number" id="water_m3" value="0" min="0">
                </div>
            </div>
        </div>

        <div class="tab-content">
            <h2>💰 Optimizer Settings</h2>
            <div class="form-row">
                <div class="form-group">
                    <label for="capex_budget">CAPEX Budget (EUR)</label>
                    <input type="number" id="capex_budget" value="0" min="0">
                </div>
                <div class="form-group">
                    <label for="max_payback">Max payback (years)</label>
                    <input type="number" id="max_payback" value="5" min="0" step="0.5">
                </div>
                <div class="form-group">
                    <label for="elec_price">Elec price (EUR/kWh)</label>
                    <input type="number" id="elec_price" value="0.12" min="0" step="0.01">
                </div>
                <div class="form-group">
                    <label for="fuel_price">Fuel price (EUR/L)</label>
                    <input type="number" id="fuel_price" value="1.7" min="0" step="0.1">
                </div>
            </div>
        </div>

        <div class="tab-content">
            <h2>🤖 AI Recommendations (Optional)</h2>
            <div style="margin-bottom: 20px;">
                <label><input type="checkbox" id="use_ai"> Enable AI recommendations</label>
            </div>
            <div class="form-group">
                <label for="groq_api_key">Groq API Key (free at console.groq.com)</label>
                <input type="text" id="groq_api_key" placeholder="Enter your API key..." value="">
            </div>
            <p style="color: #6c757d; margin-top: 12px;">
                Get your free API key at <a href="https://console.groq.com" target="_blank">console.groq.com</a>
            </p>
        </div>

        <div class="results" id="results">
            <div class="loading" id="loading">
                <div class="spinner"></div>
                Calculating...
            </div>
            <div id="results-content"></div>
        </div>
    </div>

    <script>
        function switchTab(index) {
            document.querySelectorAll('.tab').forEach((tab, i) => {
                tab.classList.toggle('active', i === index);
            });
            document.querySelectorAll('.tab-content').forEach((content, i) => {
                content.classList.toggle('active', i === index);
            });
        }

        function fillExample() {
            document.getElementById('electricity_kwh').value = 800000;
            document.getElementById('natural_gas_kwh').value = 400000;
            document.getElementById('diesel_liters').value = 50000;
            document.getElementById('gasoline_liters').value = 10000;
            document.getElementById('r410a_kg_leaked').value = 5;
            document.getElementById('flights_short_pkm').value = 120000;
            document.getElementById('flights_long_pkm').value = 80000;
            document.getElementById('train_pkm').value = 15000;
            document.getElementById('road_ton_km').value = 900000;
            document.getElementById('air_ton_km').value = 5000;
            document.getElementById('spend_packaging_eur').value = 250000;
            document.getElementById('spend_metals_eur').value = 500000;
            document.getElementById('spend_it_eur').value = 80000;
            document.getElementById('spend_office_eur').value = 40000;
            document.getElementById('waste_mixed_ton').value = 120;
            document.getElementById('water_m3').value = 2000;
            document.getElementById('capex_budget').value = 500000;
            alert('✅ Example data filled!');
        }

        function clearForm() {
            document.querySelectorAll('input[type="number"]').forEach(input => {
                if (input.id === 'max_payback') input.value = 5;
                else if (input.id === 'elec_price') input.value = 0.12;
                else if (input.id === 'fuel_price') input.value = 1.7;
                else input.value = 0;
            });
            document.getElementById('has_green').checked = false;
            document.getElementById('use_ai').checked = false;
            document.getElementById('results').classList.remove('show');
        }

        async function calculateFootprint() {
            const data = {
                electricity_kwh: parseFloat(document.getElementById('electricity_kwh').value) || 0,
                has_green: document.getElementById('has_green').checked,
                green_electricity_kwh: parseFloat(document.getElementById('green_electricity_kwh').value) || 0,
                natural_gas_kwh: parseFloat(document.getElementById('natural_gas_kwh').value) || 0,
                diesel_liters: parseFloat(document.getElementById('diesel_liters').value) || 0,
                gasoline_liters: parseFloat(document.getElementById('gasoline_liters').value) || 0,
                lpg_kg: parseFloat(document.getElementById('lpg_kg').value) || 0,
                r410a_kg_leaked: parseFloat(document.getElementById('r410a_kg_leaked').value) || 0,
                r134a_kg_leaked: parseFloat(document.getElementById('r134a_kg_leaked').value) || 0,
                flights_short_pkm: parseFloat(document.getElementById('flights_short_pkm').value) || 0,
                flights_long_pkm: parseFloat(document.getElementById('flights_long_pkm').value) || 0,
                train_pkm: parseFloat(document.getElementById('train_pkm').value) || 0,
                road_ton_km: parseFloat(document.getElementById('road_ton_km').value) || 0,
                sea_ton_km: parseFloat(document.getElementById('sea_ton_km').value) || 0,
                air_ton_km: parseFloat(document.getElementById('air_ton_km').value) || 0,
                spend_packaging_eur: parseFloat(document.getElementById('spend_packaging_eur').value) || 0,
                spend_metals_eur: parseFloat(document.getElementById('spend_metals_eur').value) || 0,
                spend_it_eur: parseFloat(document.getElementById('spend_it_eur').value) || 0,
                spend_office_eur: parseFloat(document.getElementById('spend_office_eur').value) || 0,
                waste_mixed_ton: parseFloat(document.getElementById('waste_mixed_ton').value) || 0,
                water_m3: parseFloat(document.getElementById('water_m3').value) || 0,
                capex_budget: parseFloat(document.getElementById('capex_budget').value) || 0,
                max_payback: parseFloat(document.getElementById('max_payback').value) || 5,
                elec_price: parseFloat(document.getElementById('elec_price').value) || 0.12,
                fuel_price: parseFloat(document.getElementById('fuel_price').value) || 1.7,
                use_ai: document.getElementById('use_ai').checked,
                groq_api_key: document.getElementById('groq_api_key').value
            };

            document.getElementById('results').classList.add('show');
            document.getElementById('loading').style.display = 'block';
            document.getElementById('results-content').innerHTML = '';

            try {
                // Step 1: Send data to Python
                await google.colab.kernel.invokeFunction('notebook.Convert', [JSON.stringify(data)], {});

                // Step 2: Get result from Python
                await new Promise(resolve => setTimeout(resolve, 100)); // Small delay
                const result = await google.colab.kernel.invokeFunction('notebook.GetResult', [], {});
                const response = JSON.parse(result.data['text/plain'].replace(/^'|'$/g, ''));

                document.getElementById('loading').style.display = 'none';

                if (response.error) {
                    document.getElementById('results-content').innerHTML =
                        `<div class="error"><h3>⚠️</h3><p>${response.error}</p></div>`;
                } else {
                    let html = `
                        <h2>📊 Carbon Footprint Results</h2>
                        <div class="results-summary">
                            <h3>Total Emissions</h3>
                            <p><strong>Location-based:</strong> ${response.total_location} tCO₂e</p>
                            <p><strong>Market-based:</strong> ${response.total_market} tCO₂e</p>
                        </div>

                        <h3>Emissions Breakdown</h3>
                        <table class="results-table">
                            <thead><tr><th>Category</th><th>Emissions</th><th>% of Total</th></tr></thead>
                            <tbody>
                    `;

                    response.breakdown.forEach(row => {
                        html += `<tr><td>${row[0]}</td><td>${row[1]}</td><td>${row[2]}</td></tr>`;
                    });

                    html += `</tbody></table>
                        <div class="recommendations">
                            <h3>💡 Recommendations</h3>
                            ${response.recommendations}
                        </div>`;

                    if (response.ai_recommendations) {
                        html += `
                            <div class="recommendations" style="background: linear-gradient(135deg, #28a745 0%, #20c997 100%); color: white; border: none;">
                                <h3 style="color: white;">🤖 AI Recommendations</h3>
                                <div style="background: white; color: #333; padding: 16px; border-radius: 8px; margin-top: 12px;">
                                    ${response.ai_recommendations}
                                </div>
                            </div>`;
                    }

                    document.getElementById('results-content').innerHTML = html;
                    document.getElementById('results').scrollIntoView({ behavior: 'smooth' });
                }
            } catch (error) {
                document.getElementById('loading').style.display = 'none';
                document.getElementById('results-content').innerHTML =
                    `<div class="error"><h3>❌ Error</h3><p>${error.message}</p>
                    <p style="margin-top: 12px;">Make sure you're running this in Google Colab!</p></div>`;
            }
        }
    </script>
</body>
</html>
"""

# Register functions for JavaScript to call
from google.colab import output

def convert_data(data_json):
    """Store data and calculate"""
    calculate_and_store(data_json)
    return ""

def get_result():
    """Return the stored result"""
    global _last_result
    return json.dumps(_last_result)

output.register_callback('notebook.Convert', convert_data)
output.register_callback('notebook.GetResult', get_result)

# Display the interface
display(HTML(html_code))

print("\n✅ Carbon Footprint Calculator loaded successfully!")
print("📝 Click 'Fill Example Data' to populate the form")
print("▶️ Then click 'Calculate Footprint' to get results")
print("🤖 Enable AI in the AI tab for intelligent recommendations (optional)")

⚙️  Carbon Footprint Calculator
📝 To enable AI recommendations (optional):
   1. Get a free API key from: https://console.groq.com
   2. Enter it in the '🤖 AI Settings' tab



✅ Carbon Footprint Calculator loaded successfully!
📝 Click 'Fill Example Data' to populate the form
▶️ Then click 'Calculate Footprint' to get results
🤖 Enable AI in the AI tab for intelligent recommendations (optional)
🔄 Calculating carbon footprint...
📊 Received data: electricity=800000 kWh
✓ Total emissions: 1,299.5 tCO2e
✓ Selected 3 reduction measures
✅ Calculation complete!
